# ANSWERS PREDICT

In [ ]:
%matplotlib inline

import os
import sys
import certifi
import unicodedata
import pandas as pd
import numpy as np
import re
import csv
import math
import requests
import time
import random
import scipy.stats as stats
import matplotlib.pyplot as plt
import gensim
import keras
import h5py
import pydot
import os
import sklearn
import operator

from keras.layers.merge import Add
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical, plot_model
from keras.layers import Conv2D, MaxPooling2D, Embedding, Merge, LSTM, Bidirectional, GlobalMaxPooling1D, Dot, Lambda, Dropout
from keras.layers import Average, dot,Permute, Lambda,Layer, add, Concatenate, Dense, LSTM, Input, concatenate, merge, Add, Reshape, Flatten
from keras.models import Model, load_model
from keras.models import Sequential
from numpy import newaxis
from keras.models import model_from_json
from keras import callbacks
from keras.callbacks import ModelCheckpoint
from collections import defaultdict
from scipy.stats.stats import pearsonr
from keras import backend as K
from sklearn.model_selection import StratifiedKFold

K.os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
K.os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [ ]:
#FUNCTON FOR MATRIX MULTIPLICATION
class OuterProduct(Layer):
    def __init__(self,name="OuterProduct",**kwargs):
        super().__init__()

    def build(self,input_shape):
        self.length = input_shape[0][1]
        super().build(input_shape)

    def call(self,x):
        a, b = x
#        print(a.shape,b.shape,a[:, newaxis, :].shape,b[:, :, newaxis].shape)
        outerProduct = a[:, newaxis, :] * b[:, :, newaxis]
 #       print(outerProduct.shape)
        outerProduct = K.sum(outerProduct,axis=-1)
  #      print(outerProduct.shape)

        return outerProduct

    def compute_output_shape(self, input_shape):
        return (input_shape[0][0], input_shape[0][1], input_shape[0][1])

    
#SUBSTITUTE WORD WITH INDEX
def replace_matched_items(array, dictionary):
       for lst in array:
              for ind, item in enumerate(lst):
                          lst[ind] = dictionary.get(item, item)
                        
#CLEAN ZEROS
def cleaningZero(array):
       for elem in array:
            while 0 in elem: 
                elem.remove(0)
                


target = 25

#SAVE ANSWER INDEXS
def getAnswerID(pred):
    x={}
    for i in range(0,len(pred)):
        x[i]=float(pred[i])
    sorted_x = sorted(x.items(), key=operator.itemgetter(1))
    sorted_x.reverse()
    return sorted_x[0:target]

def getAnswer(resp,test):
    lst = [item[0] for item in resp]
    ans = []
    for i in lst:
        ans.append(test[i])
    return ans

#SPLIT ANSWERS
def printAnswer(a):
    answers=[]
    
    for j in  range(0,len(a)):
        sentence=""
        for i in a[j]:
            sentence=sentence+" "+i
        answers.append(sentence)
    return answers

# LOAD TOKENIZER SENTENCES

In [ ]:
#LOAD TOKENIZER FILES
def loadSentences(filename):
    array = []
    with open(filename) as f:
        ar = []
        for line in f:
            if (str(line) != "[\n") & (str(line) != "]\n"):
                ar.append(re.sub('\n$', '', line))
                #print (line)
            if (str(line) == "]\n"):
                array.append(ar)
                ar = []
            
    f.close()
    return array

#ANSWERING TEST 
original_A_test = loadSentences("tokenizer/tokenizerAnswer.txt")
A_test = loadSentences("tokenizer/tokenizerAnswer.txt")

#QUESTION TEST--> 3 QUESTIONS
original_Q_test = loadSentences("tokenizer/tokenizerQTest.txt")
Q_test = loadSentences("tokenizer/tokenizerQTest.txt")

# LOAD KERAS MODELS

In [ ]:
#CONV2D - MAXPOOLING MODEL
#product = load_model("keras_models/Q_QA_Conv2D_MaxPool.h5", custom_objects={'OuterProduct': OuterProduct})

#AVERAGE BI-LSTM MODEL
average = load_model("keras_models/Q_QA_Avg.h5", custom_objects={'OuterProduct': OuterProduct})

# LOAD WORD2VEC MODEL

In [ ]:
#LOAD GENSIM MODEL WITH WORD2VEC
model_W2C = gensim.models.Word2Vec.load('model/mymodelFull')

#vocab - uso lo stesso vocab
vocab = dict([(k, v.index) for k, v in model_W2C.wv.vocab.items()])

# TEST QUESTION 1
Come posso pagare la bolletta?

In [ ]:
#GET QUESTION TEST NUMBER 1
Q_test=original_Q_test[0]
Q_test=[Q_test]*len(A_test)

In [ ]:
#REPLACE A_TEST AND Q_TEST WITH VOCAB INDEX
replace_matched_items(A_test,vocab)
cleaningZero(A_test)
replace_matched_items(Q_test,vocab)
cleaningZero(Q_test)

In [ ]:
# pad documents with original max len
max_len = max(len(max(A_test,key=len)), len(max(Q_test,key=len)))
max_len=580
Q_test_padded = keras.preprocessing.sequence.pad_sequences(Q_test, padding='post', maxlen = max_len)
A_test_padded = keras.preprocessing.sequence.pad_sequences(A_test, padding='post', maxlen = max_len)
print(Q_test_padded.shape, A_test_padded.shape)

In [ ]:
#CONV2D MAXPOOLING FOR QUESTION 1
prodQ1 = product.predict([Q_test_padded,A_test_padded])
ansID = getAnswerID(prodQ1)
AnswersConvQ1 = getAnswer(ansID,original_A_test)

In [ ]:
#AVERAGE PREDICT FOR QUESTION 1
avgQ1 = average.predict([Q_test_padded,A_test_padded])
ansID = getAnswerID(avgQ1)
AnswersAVGQ1= getAnswer(ansID,original_A_test)

In [ ]:
#SAVE CONV2D MAXPOOLING ANSWERS IN A TXT FILE
Answers=printAnswer(AnswersConvQ1)
f = open("Results/Q+QA_convutional_maxpooling", "w")
f.write("----ANSWERS FOR QUESTION 1---\n")
f.write("\n".join(map(lambda x: str(x), Answers)))

In [ ]:
#SAVE AVERAGE ANSWERS IN A TXT FILE
Answers=printAnswer(AnswersAVGQ1)
g = open("Results/Q+QA_average", "w")
g.write("----ANSWERS FOR QUESTION 1---\n")
g.write("\n".join(map(lambda x: str(x), Answers)))

# TEST QUESTION 2
Cos'è il deposito cauzionale?

In [ ]:
#GET QUESTION TEST NUMBER 1
Q_test=original_Q_test[1]
Q_test=[Q_test]*len(A_test)
print(len(Q_test))

In [ ]:
#REPLACE A_TEST AND Q_TEST WITH VOCAB INDEX
replace_matched_items(A_test,vocab)
cleaningZero(A_test)
replace_matched_items(Q_test,vocab)
cleaningZero(Q_test)

In [ ]:
# pad documents with original max len
#max_len = max(len(max(A_test,key=len)), len(max(Q_test,key=len)))
Q_test_padded = keras.preprocessing.sequence.pad_sequences(Q_test, padding='post', maxlen = max_len)
A_test_padded = keras.preprocessing.sequence.pad_sequences(A_test, padding='post', maxlen = max_len)
print(Q_test_padded.shape, A_test_padded.shape)

In [ ]:
#CONV2D MAXPOOLING FOR QUESTION 2
prodQ2 = product.predict([Q_test_padded,A_test_padded])
ansID = getAnswerID(prodQ2)
AnswersConvQ2 = getAnswer(ansID,original_A_test)

In [ ]:
#AVERAGE PREDICT FOR QUESTION 2
avgQ2 = average.predict([Q_test_padded,A_test_padded])
ansID = getAnswerID(avgQ1)
AnswersAVGQ2= getAnswer(ansID,original_A_test)

In [ ]:
#SAVE CONV2D MAXPOOLING ANSWERS IN A TXT FILE
Answers=printAnswer(AnswersConvQ2)
f.write("\n----ANSWERS FOR QUESTION 2---\n")
f.write("\n".join(map(lambda x: str(x), Answers)))

In [ ]:
#SAVE AVERAGE ANSWERS IN A TXT FILE
Answers=printAnswer(AnswersAVGQ2)
g.write("\n----ANSWERS FOR QUESTION 2---\n")
g.write("\n".join(map(lambda x: str(x), Answers)))

# TEST QUESTION 3
Ho bisogno di aiuto

In [ ]:
#GET QUESTION TEST NUMBER 1
Q_test=original_Q_test[2]
Q_test=[Q_test]*len(A_test)
print(len(Q_test))

In [ ]:
#REPLACE A_TEST AND Q_TEST WITH VOCAB INDEX
replace_matched_items(A_test,vocab)
cleaningZero(A_test)
replace_matched_items(Q_test,vocab)
cleaningZero(Q_test)

In [ ]:
# pad documents with original max len
#max_len=580
#max_len = max(len(max(A_test,key=len)), len(max(Q_test,key=len)))
Q_test_padded = keras.preprocessing.sequence.pad_sequences(Q_test, padding='post', maxlen = max_len)
A_test_padded = keras.preprocessing.sequence.pad_sequences(A_test, padding='post', maxlen = max_len)
print(Q_test_padded.shape, A_test_padded.shape)

In [ ]:
# CONV2D MAXPOOLING FOR QUESTION 3
prodQ3 = product.predict([Q_test_padded,A_test_padded])
ansID = getAnswerID(prodQ3)
AnswersConvQ3 = getAnswer(ansID,original_A_test)

In [ ]:
#AVERAGE PREDICT FOR QUESTION 3
avgQ3 = average.predict([Q_test_padded,A_test_padded])
ansID = getAnswerID(avgQ3)
AnswersAVGQ3= getAnswer(ansID,original_A_test)

In [ ]:
#SAVE CONV2D MAXPOOLING ANSWERS IN A TXT FILE AND CLOSE CONNECTION
Answers=printAnswer(AnswersConvQ3)
f.write("\n----ANSWERS FOR QUESTION 3---\n")
f.write("\n".join(map(lambda x: str(x), Answers)))
f.close()

In [ ]:
#SAVE AVERAGE ANSWERS IN A TXT FILE AND CLOSE CONNECTION
Answers=printAnswer(AnswersAVGQ3)
g.write("\n----ANSWERS FOR QUESTION 3---\n")
g.write("\n".join(map(lambda x: str(x), Answers)))
g.close()

# EXPERIMENTS

In [ ]:
#domanda1=['Ho','rilevato','un','consumo','anomalo'] #OK
#domanda2=['Qual','è','il','costo','acqua'] #OK
domanda3=['Come','posso','richiedere','il','bonus','idrico']

In [ ]:
#GET QUESTION TEST NUMBER 1
Q_test=domanda3
Q_test=[Q_test]*len(A_test)
print(len(Q_test))

In [ ]:
#REPLACE A_TEST AND Q_TEST WITH VOCAB INDEX
replace_matched_items(A_test,vocab)
cleaningZero(A_test)
replace_matched_items(Q_test,vocab)
cleaningZero(Q_test)

In [ ]:
# pad documents with original max len
max_len = max(len(max(A_test,key=len)), len(max(Q_test,key=len)))
Q_test_padded = keras.preprocessing.sequence.pad_sequences(Q_test, padding='post', maxlen = max_len)
A_test_padded = keras.preprocessing.sequence.pad_sequences(A_test, padding='post', maxlen = max_len)
print(Q_test_padded.shape, A_test_padded.shape)

In [ ]:
#CONV2D MAXPOOLING FOR QUESTION 3
prodQ = product.predict([Q_test_padded,A_test_padded])
ansID = getAnswerID(prodQ)
Answers = getAnswer(ansID,original_A_test)

In [ ]:
ansID

In [ ]:
print(Answers[0])